# Project milestone 4
Tadaa - Jonathan Haenni, Lea Schmidt, Danny Kohler
> This work presents the creative extension of the publication from J. Penney : "Chilling effects : Online surveillance and Wikipedia use".

---

## Abstract
The goal of this work is to observe if the public action of Greta Thunberg and her continual incitation for people to educate themselves on the subject of climate change has actually made a difference. This effect would be diametrically different from a chilling effect in that it pushes people to get educated on a subject, effectively arguably increasing their levels of freedom. This effect shall henceforth be called an "Empowering effect".

First and foremost, the selected interest period depends on the choice of events that will be the pivots of the interrupted time series analysis. (ITS) These events are as follows: the school strike of August 20, 2018 led by Greta Thunberg and the 2018 United Nations Climate Change Conference (COP24) on December 14, 2018. The period of analysis is between January 2018 and February 2020. As seen in HW1, the Covid-19 pandemic influences the pageviews a lots and we will therefore not elongate this analysis onto 2020.

The data considered here comes from Wikipedia. We will consider 150 [ADAPT VALUE] Wikipedia articles divided into 3 groups. The first group is the treatment dataset, containing the Wikipedia articles related to climate change issues. The second group is a quasi-control group, where the topics considered are related to nature, without being related to climate change directly. The third group is another control group composed of popular articles simply reflecting the trends on Wikipedia. It is the same as the one used in the publication. The data sets considered give the number of pageviews per day for each article, within the time period considered.

As already hinted at, the analysis will be very similar to the one used in the publication - an ITS analysis with segmented linear regressions.

If the first group features significant changes in pageviews and the control groups don't across the selected period, we will be able to conclude that the "Empowering Effect" exists and we will be able to compare it to the chilling effect considered in the publication.

---

## 1. Data loading and filtering

> code description

In [1]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt

# Load the climate change articles data
art_cc1 = pd.read_csv('data/pageviews-10-pro.csv').copy()
art_cc2 = pd.read_csv('data/pageviews-20-pro.csv').copy()
art_cc3 = pd.read_csv('data/pageviews-30-pro.csv').copy()
art_cc4 = pd.read_csv('data/pageviews-40-pro.csv').copy()
art_cc5 = pd.read_csv('data/pageviews-50-pro.csv').copy()

# Load the control (popular) articles data
art_control1 = pd.read_csv('data/pageviews-10-control.csv').copy()
art_control2 = pd.read_csv('data/pageviews-20-control.csv').copy()
art_control3 = pd.read_csv('data/pageviews-30-control.csv').copy()
art_control4 = pd.read_csv('data/pageviews-40-control.csv').copy()
#art_control5 = pd.read_csv('data/pageviews-50-control.csv').copy()

# Merge all datasets together
art_cc=art_cc1.merge(art_cc2).merge(art_cc3).merge(art_cc4).merge(art_cc5)
art_control=art_control1.merge(art_control2).merge(art_control3).merge(art_control4)#.merge(art_control5)

# Display to illustrate the structure
art_control.head()

,Date,Keanu Reeves,Freddie Mercury,List of highest-grossing films,Ted Bundy,Chernobyl disaster,Joker,List of Marvel Cinematic Universe films,Avengers: Endgame,Billie Eilish,...,India,Michael Jordan,The Beatles,Venom,Barack Obama,New York City,Jason Momoa,Facebook,Millie Bobby Brown,Once Upon a Time in Hollywood
0,2018-01-01,23376,41612,39321,12301,6855,176,33525,38,3071,...,25716,14932,14694,407,20396,29579,22491,23951,23436,0
1,2018-01-02,18217,17525,34241,11654,8369,219,33360,10,3489,...,32794,15537,12617,528,19637,19933,20131,25019,23641,0
2,2018-01-03,24872,12716,29450,12447,8957,209,29134,11,3751,...,30876,16124,12564,503,22417,20296,20808,27744,24889,0
3,2018-01-04,18654,12621,27118,11718,11888,178,27534,17,4162,...,31401,18752,12715,498,20881,18112,19602,25977,20850,0
4,2018-01-05,17674,12407,25787,10768,12103,240,28231,19,4069,...,29565,21127,24112,465,20061,17256,21141,33260,20353,0


In [2]:
# Translate dates in datetime type
art_cc.Date=pd.to_datetime(art_cc.Date)
art_control.Date=pd.to_datetime(art_control.Date)

# Set the date as the index
art_cc=art_cc.set_index('Date')
art_control=art_control.set_index('Date')

# Agregate the data by months
art_cc=art_cc.groupby(pd.Grouper(freq="M")).sum()
art_control=art_control.groupby(pd.Grouper(freq="M")).sum()

# Display to illustrate the structure
art_control.head()

,Keanu Reeves,Freddie Mercury,List of highest-grossing films,Ted Bundy,Chernobyl disaster,Joker,List of Marvel Cinematic Universe films,Avengers: Endgame,Billie Eilish,Deaths in 2019,...,India,Michael Jordan,The Beatles,Venom,Barack Obama,New York City,Jason Momoa,Facebook,Millie Bobby Brown,Once Upon a Time in Hollywood
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-31,684194,462645,716662,727098,288451,6649,817572,643,96218,0,...,1003610,572133,413801,16498,919624,553957,798788,1358403,945691,46
2018-02-28,708393,347784,692353,546928,272240,5806,1313523,542,72000,1,...,874309,812053,381631,20616,840704,473983,604739,953859,592593,21
2018-03-31,581780,497950,1376046,403032,331912,6378,2011814,1072,103930,0,...,967881,685320,457535,17332,704845,533678,626376,1150844,303550,105517
2018-04-30,557361,385714,1665722,376607,353361,5210,2947190,5087,122300,0,...,886126,625658,370273,28706,698878,534083,481257,1117173,315895,95950
2018-05-31,561971,1207507,3127269,341641,300443,4694,3604438,19526,172702,0,...,940868,1010579,370202,24160,660080,541006,436300,974303,258842,55197


In [3]:
# At this point, we should  sum over all articles.
art_cc=art_cc.sum(axis=1)
art_control=art_control.sum(axis=1)

# Here the index is reset to get the number of months instead of dates, as in the publication.
# The date column is then removed : it is not needed anymore
# The pageviews column is renamed
art_cc=art_cc.reset_index()
art_cc=art_cc.drop('Date', axis=1)
art_cc.columns=['pageviews']

art_control=art_control.reset_index()
art_control=art_control.drop('Date', axis=1)
art_control.columns=['pageviews']

# Display to illustrate the structure
art_control.head()

,pageviews
0,31124431
1,35708117
2,45202785
3,43051638
4,69103350


---

## 2. Computing the Segmented Regression parameters

> code description

In [4]:
#CODE

---

## 3. Displaying the results

> code description

In [5]:
#CODE